# Calculate SI from seasonal fire mask and NOT fire mask reflectance Data


This notebook calculates the Spectral Indices of the dbi features and append the data to the end of the df.

The following conditions apply:

 - env = base
 - Run following: collate_met_and_seasonal_data_fire_mask_and_not.ipynb


In [1]:
index_ = 11

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('chained_assignment',None)
from __future__ import division
import os
from glob import glob
#pd.set_option('precision', 8)

In [3]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
date_str = now.strftime("%Y%m%d")
date_time_str = now.strftime("%Y%m%d_%H%M%S")
print(date_str)
print(date_time_str)

20230205
20230205_085716


In [4]:
drive = "D"

# # data date
# date = "20230204"

# define output directory
output_dir = r"{0}:\cdu\data\zonal_stats\output\{1}".format(drive, date_str)

# data dir
dir_ = r"{0}:\cdu\data\zonal_stats\output\{1}\ml_data_dir".format(drive, date_str)



In [5]:
def mk_dir_fn(dir_):
    """ Create a new directory if one does not already exist. """
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

        
def export_csv_fn(list_, dir_, file_name):
    
    """ Create and export path from directory and file name and exports csv with no dropping the index column. """
    
    df_final = pd.concat(list_, axis =0)    
    output_path = os.path.join(dir_, file_name)
    df_final.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
    
    return df_final

In [6]:
##### Create output directories

In [7]:
ml_data_dir = os.path.join(output_dir, "ml_data_si_dir")
mk_dir_fn(output_dir)
mk_dir_fn(ml_data_dir)

In [8]:
# this note book will take in the combined training and validation data which has been combined and produces the 
# 
# ratios and vegetation indices for Landsat-8 (use the one written for the sentinel-2) and converts them to int32 bit data 
# 
# and reduces the significant figures to 8 decimal places. Comment out the relevent input csv file at the start and finish
#

In [9]:
# read in the zonal stats results
#df = pd.read_csv(r"{0}:\cdu\data\zonal_stats\output\{1}\dja_dbi_dim_dis_dka_stc_h99a2_fpca2_dry_clean_df.csv".format(drive, date), header=0)
#path_csv = r"{0}:\cdu\data\zonal_stats\output\{1}\ml_data_dir\r2_best_season_climate_fillna_fms_applied_reg.csv".format(drive, date)


file_list = []
for f in glob(os.path.join(dir_, "*.csv")):
    print(f)
    file_list.append(f)


D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std__fnm_fms_applied_class.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std__fnm_fms_NOT_applied_class.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\r2_bs_mean_max_min_med_std__fnm_fms_NOT_applied_class.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std_met_fnm_fms_NOT_applied_class.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\r2_bs_mean_max_min_med_std_met_fnm_fms_NOT_applied_class.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std__fnm_fms_applied_reg.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\r2_bs_mean_max_min_med_std__fnm_fms_applied_reg.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std_met_fnm_fms_applied_reg.csv
D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\r2_bs_mean_max_min_med_std_met_fnm_fms_applied_reg.csv
D:\cdu\data\zonal_stats\output\20230205\ml_dat

In [10]:
print(f"Working on  {file_list[index_]}")
df = pd.read_csv(file_list[index_], header=0)
print (list(df))
print (df.shape)

IndexError: list index out of range

In [ ]:
df

In [ ]:
print(list(df.columns))

In [ ]:
a = df.copy(deep=True)

In [ ]:
a.info()

In [ ]:
a

In [ ]:
def select_dbi_fn(df):
    """ Remove features from dataframe """
    df_columns = df.columns.tolist()
    print(f"Initial columns: {df_columns}")
    print("-"*100)
    drop_list = [columns for columns in df_columns if 'dbi_0608_mean' not in columns]
    print(drop_list)
    drop_list.remove("site")

    df.drop(drop_list, axis = 1, inplace=True)
    
        
    return df, drop_list

In [ ]:
a, drop_list = select_dbi_fn(a)

In [ ]:
print(list(a))

In [ ]:
a.columns = ["site", "psB1a", "psB2a", "psB3a", "psB4a", "psB5a", "psB6a", ]

In [ ]:
a.dropna(inplace=True)

In [ ]:
a.info()

In [ ]:
# read in the zonal stats results
#a = pd.read_csv("combined_val_data_l8.csv", header=0)
#print (list(a))

In [ ]:
# converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# to floating point for analysis of the vegetation index, blue band not used

blue =((a['psB1a'].astype('int16')*0.0001)+0.0)

green =((a['psB2a'].astype('int16')*0.0001)+0.0)
red = ((a['psB3a'].astype('int16')*0.0001)+0.0)
nir = ((a['psB4a'].astype('int16')*0.0001)+0.0)
swir1 =((a['psB5a'].astype('int16')*0.0001)+0.0)
swir2 =((a['psB6a'].astype('int16')*0.0001)+0.0)

In [ ]:
# calculate the band ratios

a['ratio32fa'] = (a['psB3a'] / a['psB2a'])
a['ratio42fa'] = (a['psB4a'] / a['psB2a'])
a['ratio43fa'] = (a['psB4a'] / a['psB3a'])
a['ratio52fa'] = (a['psB5a'] / a['psB2a'])
a['ratio53fa'] = (a['psB5a'] / a['psB3a'])
a['ratio54fa'] = (a['psB5a'] / a['psB4a'])
a['ratio62fa'] = (a['psB6a'] / a['psB2a'])
a['ratio63fa'] = (a['psB6a'] / a['psB3a'])
a['ratio64fa'] = (a['psB6a'] / a['psB4a'])
a['ratio65fa'] = (a['psB6a'] / a['psB5a'])

# calculate the band ratios and convert them to int32 bit at 7 decimal places

a['ratio32a'] = np.int32(np.around(a['ratio32fa'] * 10 ** 7))

a['ratio42a'] = np.int32(np.around(a['ratio42fa'] * 10 ** 7))

a['ratio43a'] = np.int32(np.around(a['ratio43fa'] * 10 ** 7))

a['ratio52a'] = np.int32(np.around(a['ratio52fa'] * 10 ** 7))

a['ratio53a'] = np.int32(np.around(a['ratio53fa'] * 10 ** 7))

a['ratio54a'] = np.int32(np.around(a['ratio54fa'] * 10 ** 7))

a['ratio62a'] = np.int32(np.around(a['ratio62fa'] * 10 ** 7))

a['ratio63a'] = np.int32(np.around(a['ratio63fa'] * 10 ** 7))

a['ratio64a'] = np.int32(np.around(a['ratio64fa'] * 10 ** 7))

a['ratio65a'] = np.int32(np.around(a['ratio65fa'] * 10 ** 7))

In [ ]:
# calculate the vegetation indices
a['GSAVIfa'] = ((nir-green)/(nir+green+0.5))*(1.5)
a['GSAVIa'] = np.int32(np.around(a['GSAVIfa'] * 10 ** 7))

a['GNDVIfa'] = (nir-green)/(nir+green)
a['GNDVIa'] = np.int32(np.around(a['GNDVIfa'] * 10 ** 7))

a['CVIfa']= (nir/green)*(red/green)
a['CVIa'] = np.int32(np.around(a['CVIfa'] * 10 ** 7))

a['NDGIfa']= (green-red)/(green+red)
a['NDGIa'] = np.int32(np.around(a['NDGIfa'] * 10 ** 7))

a['RIfa']= (red-green)/(red+green)
a['RIa'] = np.int32(np.around(a['RIfa'] * 10 ** 7))

a['NBRfa']= (nir-swir2)/(nir+swir2)
a['NBRa'] = np.int32(np.around(a['NBRfa'] * 10 ** 7))

a['NDIIfa']= (nir-swir1)/(nir+swir1)
a['NDIIa'] = np.int32(np.around(a['NDIIfa'] * 10 ** 7))

a['GDVIfa']= (nir-green)
a['GDVIa'] = np.int32(np.around(a['GDVIfa'] * 10 ** 7))

a['MSAVIfa'] = (2 * nir + 1 - np.sqrt((np.power(((2*nir)+1) , 2)) - (8 * (nir-red))))/2
a['MSAVIa'] = np.int32(np.around(a['MSAVIfa'] * 10 ** 7))

a['DVIfa'] = (nir-red)
a['DVIa'] = np.int32(np.around(a['DVIfa'] * 10 ** 7))

a['SAVIfa']= ((nir-red)/(nir+red+0.5))*(1+0.5)
a['SAVIa'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

a['NDVIfa']= (nir-red)/(nir+red)
a['NDVIa'] = np.int32(np.around(a['NDVIfa'] * 10 ** 7))

a['MSRfa']= (((nir/red)-1)/((np.sqrt(nir/red))+1))
a['MSRa'] = np.int32(np.around(a['MSRfa'] * 10 ** 7))

In [ ]:
a

In [ ]:
a.info()

In [ ]:
# # converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# # to floating point for analysis of the vegetation index, blue band not used

# blued =((a['psB1d'].astype('int16')*0.0001)+0.0)

# greend =((a['psB2d'].astype('int16')*0.0001)+0.0)
# redd = ((a['psB3d'].astype('int16')*0.0001)+0.0)
# nird = ((a['psB4d'].astype('int16')*0.0001)+0.0)
# swir1d =((a['psB5d'].astype('int16')*0.0001)+0.0)
# swir2d =((a['psB6d'].astype('int16')*0.0001)+0.0)

In [ ]:
# # calculate the band ratios

# a['ratio32fd'] = (a['psB3d'] / a['psB2d'])
# a['ratio42fd'] = (a['psB4d'] / a['psB2d'])
# a['ratio43fd'] = (a['psB4d'] / a['psB3d'])
# a['ratio52fd'] = (a['psB5d'] / a['psB2d'])
# a['ratio53fd'] = (a['psB5d'] / a['psB3d'])
# a['ratio54fd'] = (a['psB5d'] / a['psB4d'])
# a['ratio62fd'] = (a['psB6d'] / a['psB2d'])
# a['ratio63fd'] = (a['psB6d'] / a['psB3d'])
# a['ratio64fd'] = (a['psB6d'] / a['psB4d'])
# a['ratio65fd'] = (a['psB6d'] / a['psB5d'])

# # calculate the band ratios and convert them to int32 bit at 7 decimal places

# a['ratio32d'] = np.int32(np.around(a['ratio32fd'] * 10 ** 7))

# a['ratio42d'] = np.int32(np.around(a['ratio42fd'] * 10 ** 7))

# a['ratio43d'] = np.int32(np.around(a['ratio43fd'] * 10 ** 7))

# a['ratio52d'] = np.int32(np.around(a['ratio52fd'] * 10 ** 7))

# a['ratio53d'] = np.int32(np.around(a['ratio53fd'] * 10 ** 7))

# a['ratio54d'] = np.int32(np.around(a['ratio54fd'] * 10 ** 7))

# a['ratio62d'] = np.int32(np.around(a['ratio62fd'] * 10 ** 7))

# a['ratio63d'] = np.int32(np.around(a['ratio63fd'] * 10 ** 7))

# a['ratio64d'] = np.int32(np.around(a['ratio64fd'] * 10 ** 7))

# a['ratio65d'] = np.int32(np.around(a['ratio65fd'] * 10 ** 7))

In [ ]:
# # calculate the vegetation indices
# a['GSAVIfd'] = ((nird-greend)/(nird+greend+0.5))*(1.5)
# a['GSAVId'] = np.int32(np.around(a['GSAVIfd'] * 10 ** 7))

# a['GNDVIfd'] = (nird-greend)/(nird+greend)
# a['GNDVId'] = np.int32(np.around(a['GNDVIfd'] * 10 ** 7))

# a['CVIfd']= (nird/greend)*(redd/greend)
# a['CVId'] = np.int32(np.around(a['CVIfd'] * 10 ** 7))

# a['NDGIfd']= (greend-redd)/(greend+redd)
# a['NDGId'] = np.int32(np.around(a['NDGIfd'] * 10 ** 7))

# a['RIfd']= (redd-greend)/(redd+greend)
# a['RId'] = np.int32(np.around(a['RIfd'] * 10 ** 7))

# a['NBRfd']= (nird-swir2d)/(nird+swir2d)
# a['NBRd'] = np.int32(np.around(a['NBRfd'] * 10 ** 7))

# a['NDIIfd']= (nird-swir1d)/(nird+swir1d)
# a['NDIId'] = np.int32(np.around(a['NDIIfd'] * 10 ** 7))

# a['GDVIfd']= (nird-greend)
# a['GDVId'] = np.int32(np.around(a['GDVIfd'] * 10 ** 7))

# a['MSAVIfd'] = (2 * nird + 1 - np.sqrt((np.power(((2*nird)+1) , 2)) - (8 * (nird-redd))))/2
# a['MSAVId'] = np.int32(np.around(a['MSAVIfd'] * 10 ** 7))

# a['DVIfd'] = (nird-redd)
# a['DVId'] = np.int32(np.around(a['DVIfd'] * 10 ** 7))

# a['SAVIfd']= ((nird-redd)/(nird+redd+0.5))*(1+0.5)
# a['SAVId'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

# a['NDVIfd']= (nird-redd)/(nird+redd)
# a['NDVId'] = np.int32(np.around(a['NDVIfd'] * 10 ** 7))

# a['MSRfd']= (((nird/redd)-1)/((np.sqrt(nird/redd))+1))
# a['MSRd'] = np.int32(np.around(a['MSRfd'] * 10 ** 7))

In [ ]:
print (list(a))

In [ ]:
# remove fa
tr = a[['site',
        'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 
        'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 
        'NBRa', 'NDIIa', 'GDVIa',  'MSAVIa', 'DVIa',  'SAVIa', 'NDVIa',  'MSRa']]

In [ ]:
print(list(tr))

In [ ]:
df_si = pd.merge(df, tr,  how='left', on=['site'])

In [ ]:
column_to_move = df_si.pop("target")

# insert column with insert(location, column_name, column_value)

df_si.insert(0, "target", column_to_move)

In [ ]:
dict_ = {
    'ratio32a':'ratio32m', 'ratio42a': 'ratio42m',
    'ratio43a': 'ratio43m', 'ratio52a': 'ratio52m',
    'ratio53a': 'ratio53m', 'ratio54a': 'ratio54m',
    'ratio62a': 'ratio62m', 'ratio63a': 'ratio63m',
    'ratio64a': 'ratio64m', 'ratio65a': 'ratio65m',
    'GSAVIa': 'GSAVIm', 'GNDVIa': 'GNDVIm',
    'CVIa': 'CVIm', 'NDGIa': 'NDGIm', 
    'RIa': 'RIm', 'NBRa': 'NBRm', 
    'NDIIa': 'NDIIm', 'GDVIa': 'GDVIm', 
    'MSAVIa': 'MSAVIm', 'DVIa': 'DVIm', 
    'SAVIa': 'SAVIm', 'NDVIa': 'NDVIm', 'MSRa': 'MSRm'
}

In [ ]:
df_si.rename(columns=dict_, inplace=True)

In [ ]:
# ml_data_dir

In [ ]:
_, file_name = os.path.split(file_list[index_])
file_name_split = file_name.split("_applied")
new_name = file_name_split[0] + "_si" + file_name_split[1]
path_ = os.path.join(ml_data_dir, new_name)      
df_si.to_csv(path_, index=False)
print(path_)